In [1]:
!pip install torch
!pip install tokenizers
!pip install transformers

# Pre-training BERT on Youtube Comments Dataset

In [2]:
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertForMaskedLM
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
gb_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv', on_bad_lines='skip')
gb_comments_df.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [4]:
us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')
us_comments_df.head()

/tmp/ipykernel_31/3472227992.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [5]:
comments_df = pd.concat([gb_comments_df, us_comments_df], axis=0)
print("Len of GB data - ", len(gb_comments_df))
print("Len of US data - ", len(us_comments_df))
print("Len of combined data - ", len(comments_df))
comments_df.head()

Len of GB data -  718452
Len of US data -  691400
Len of combined data -  1409852


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [6]:
len(comments_df.comment_text.unique())

622630

In [7]:
comments = comments_df.comment_text.unique()
comments[:5]

array(["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
       'To be there with a samsung phone\\n😂😂😂',
       'Thank gosh, a place I can watch it without having to be at HD... my speed doesn’t support HD',
       'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
       'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.'],
      dtype=object)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

Device:  cuda


In [38]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

model.to(device)

text = ("After Abraham Lincoln won the November 1860 presidential "
        "election on an anti-slavery platform, an initial seven "
        "slave states declared their secession from the country "
        "to form the Confederacy. War broke out in April 1861 "
        "when secessionist forces attacked Fort Sumter in South "
        "Carolina, just over a month after Lincoln's "
        "inauguration.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
inputs = tokenizer(text, return_tensors='pt')

In [40]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [41]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [42]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

Masking 15% of the tokens

In [43]:
rand = torch.rand(inputs.input_ids.shape)
# where the random array is less than 0.15, we set true
mask_arr = rand < 0.15
mask_arr

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True, False, False, False, False,  True,
         False,  True, False, False,  True,  True, False, False, False, False,
         False, False, False, False, False, False, False,  True, False, False,
         False,  True, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False,  True,
          True, False]])

We don’t want to place a MASK token over other special tokens such as CLS or SEP tokens (101 and 102 respectively).

So, we need to add an additional condition. A check for positions containing the token ids 101 or 102.

In [44]:
(inputs.input_ids != 101) * (inputs.input_ids != 102)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False]])

In [45]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True, False, False, False, False,  True,
         False,  True, False, False,  True,  True, False, False, False, False,
         False, False, False, False, False, False, False,  True, False, False,
         False,  True, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False,  True,
          True, False]])

Getting indices to be masked

In [46]:
# create selection from mask_arr
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

[12, 13, 14, 19, 21, 24, 25, 37, 41, 52, 59, 60]

masking

In [47]:
inputs.input_ids[0, selection] = 103 # mask token = 103

In [48]:
inputs.input_ids

tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,   103,   103,   103,  4132,  1010,  2019,  3988,   103,
          6658,   103,  4161,  2037,   103,   103,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,   103,  6863,  2043,
         22965,   103,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,   103,  1037,  3204,  2044,  5367,  1005,  1055,   103,
           103,   102]])

In [49]:
outputs = model(**inputs)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
outputs.keys()

In [50]:
outputs.loss

tensor(0.7097, grad_fn=<NllLossBackward0>)

## Training

In [51]:
from tqdm import notebook
from nltk.tokenize import RegexpTokenizer
import re

In [52]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    cleantext = re.sub(r'http\S+', '',cleantext)
    return cleantext

In [53]:
def clean(sentences):
    cleaned_sentences = []
    for sentence in notebook.tqdm(sentences):
        cleaned_sentences.append(preprocess(sentence))
    return cleaned_sentences

In [54]:
text = clean(comments)
text[:5]

  0%|          | 0/622630 [00:00<?, ?it/s]

["it's more accurate to call it the m+ (1000) because the price is closer than calling it the x (10).",
 'to be there with a samsung phone\\n😂😂😂',
 'thank gosh, a place i can watch it without having to be at hd... my speed doesn’t support hd',
 'what happened to the home button on the iphone x? *****cough****copying samsung******cough',
 'power is the disease.\xa0 care is the cure.\xa0 keep caring for yourself and others as best as you can.\xa0 this is life.']

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

In [ ]:
inputs.shape